# Comparative evaluation of surgical methods in the treatment of chronic irreparable supraspinatus tears: a biomechanical study

### **Authors**

**E. Koumantou**$^{a,*}$, **C. J. Feroussis**$^{b}$, **E. D. Pasiou**$^{a}$, **S. K. Kourkoulis**$^{a}$

### **Affiliations**

> **a** National Technical University of Athens, School of Applied Mathematical and Physical Sciences, Department of Mechanics, Laboratory of Biomechanics and Biomedical Physics, Iroon Polytechniou 5, 15773 Zografou, Greece
>
> **b** National and Kapodistrian University of Athens, School of Medicine, Mikras Asias 75, Athens, Greece

---

## **Abstract**

Chronic irreparable supraspinatus (SSP) tears are frequently treated using tensor fascia lata (TFL); however, these grafts are prone to recurrent ruptures. A TFL scaffold with seeded mesenchymal stem cells (MSCs) may increase the mechanical strength of the regenerated tendon. In this experimental study the above hypothesis is evaluated and the healing qualities of the MSC embedded scaffold are compared against the ones repaired with only a TFL allograft, from a biomechanical perspective. A number of skeletally mature male rabbits were used to create a chronic retracted SSP tear model. A tendon defect was created at the right shoulder of each specimen and was reconstructed six weeks later using a TFL allograft either with or without MSCs embedded. The rabbits were sacrificed twelve weeks after the operation and their biomechanical evaluation was based on the tendon's ultimate failure load and stiffness of each group. The results of the biomechanical analysis revealed that the MSC group demonstrated a higher tensile strength than the TFL group. The values of the mean ultimate failure load and the mean stiffness of both TFL and MSC groups were significantly lower compared to the respective values obtained for the intact rotator cuff (RC) group. It seems that bone marrow MSCs increased the mechanical strength of the supraspinatus tendon; however, the stiffness was similar between the operated groups in rabbit models. Based on the results, it could be stated that rotator cuff regeneration using MSCs appears as a promising approach; nonetheless, additional clinical evidence is required, before definite conclusions are drawn.

---

## Cell 1: Setup, Configuration & Core Functions

This is a **self-contained** notebook. All configuration and analysis functions are defined inline.

**Configuration:**
- Group assignments (TFL / MSC) are hardcoded below
- Data is read from `Selected_data/` (relative to this notebook)
- Results are saved to `Results/`

**Analysis pipeline:**
- Filename parsing and sample classification
- Data validation and normalization
- Stiffness calculation via sliding-window linear regression
- Energy (area under curve) via trapezoidal rule
- Group statistics and visualizations

In [ ]:
# Uncomment to install dependencies if needed
# %pip install plotly pandas numpy matplotlib scikit-learn

In [ ]:
import re
from pathlib import Path
from typing import Dict, List, Optional, Tuple

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# ============================================================================
# CONFIGURATION (hardcoded — no external config.json needed)
# ============================================================================

NOTEBOOK_DIR = Path('.').resolve()
DATA_FOLDER = NOTEBOOK_DIR / 'Selected_data'
RESULTS_FOLDER = NOTEBOOK_DIR / 'Results'
RESULTS_FOLDER.mkdir(parents=True, exist_ok=True)

# Treatment group assignments
TFL_IDS = ['B1', 'B10', 'B11', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8']
MSC_IDS = ['B5', 'B6', 'B7', 'B9', 'C1', 'C2', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15']

# Analysis parameters
STIFFNESS_R2_THRESHOLD = 0.99
STIFFNESS_WINDOW_FRACTION = 0.1
STIFFNESS_MIN_WINDOW = 5

# ============================================================================
# SAMPLE SELECTION
# ============================================================================
# All available sample IDs (for reference):
#   B-series:  B1, B5, B6, B7, B9, B10, B11
#   C-series:  C1, C2
#   D-series:  D1, D2, D3, D4, D5, D6, D7, D8, D9, D10, D11, D12, D13, D14, D15
#
# To EXCLUDE specific samples, add their IDs to the list below.
# Example: EXCLUDED_SAMPLES = ['B1', 'D15', 'C2']
# To include all samples, leave the list empty: EXCLUDED_SAMPLES = []

EXCLUDED_SAMPLES: list = []

# Derive the active sample lists (automatically filters out excluded samples)
ACTIVE_TFL = [s for s in TFL_IDS if s not in EXCLUDED_SAMPLES]
ACTIVE_MSC = [s for s in MSC_IDS if s not in EXCLUDED_SAMPLES]

if EXCLUDED_SAMPLES:
    print(f"⚠ Excluded samples: {EXCLUDED_SAMPLES}")
    print(f"  Active TFL IDs ({len(ACTIVE_TFL)}): {ACTIVE_TFL}")
    print(f"  Active MSC IDs ({len(ACTIVE_MSC)}): {ACTIVE_MSC}")
else:
    print("All samples included (no exclusions).")

# Optional: Interactive plotting with Plotly
PLOTLY_AVAILABLE = False
try:
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    import plotly.express as px
    PLOTLY_AVAILABLE = True
    print("Plotly loaded - interactive plots enabled!")
except ImportError:
    print("Plotly not available. Install with: pip install plotly")
    print("Falling back to matplotlib (non-interactive) plots.")

# Set plot style
try:
    plt.style.use('seaborn-v0_8-whitegrid')
except OSError:
    try:
        plt.style.use('seaborn-whitegrid')
    except OSError:
        pass  # Use default style
plt.rcParams['figure.figsize'] = [10, 6]

print(f"Data folder: {DATA_FOLDER}")
print(f"Results folder: {RESULTS_FOLDER}")
print(f"\nActive TFL IDs ({len(ACTIVE_TFL)}): {ACTIVE_TFL}")
print(f"Active MSC IDs ({len(ACTIVE_MSC)}): {ACTIVE_MSC}")
print(f"Total active samples: {len(ACTIVE_TFL) + len(ACTIVE_MSC)}")
print(f"\nAnalysis parameters:")
print(f"  R² threshold: {STIFFNESS_R2_THRESHOLD}")
print(f"  Window fraction: {STIFFNESS_WINDOW_FRACTION}")
print(f"  Min window: {STIFFNESS_MIN_WINDOW}")


# ============================================================================
# FILENAME PARSING & CLASSIFICATION (from utils.py)
# ============================================================================

def parse_filename(filename: str) -> Tuple[Optional[str], str]:
    """
    Parse an SSP data filename to extract subject ID and condition.
    Returns (subject_id, condition).
    """
    condition = 'Unknown'
    if '_NO' in filename or '_NO.' in filename:
        condition = 'NO'
    elif '_OPER' in filename:
        condition = 'OPER'

    subject_id = None
    parts = filename.replace('.csv', '').split('_')
    for part in parts:
        if re.match(r'^[BCD]\d{1,2}$', part):
            subject_id = part
            break

    return subject_id, condition


def classify_sample(
    filename: str,
    tfl_ids: List[str],
    msc_ids: List[str]
) -> Tuple[str, str, str]:
    """
    Parse filename to find Sample ID, Condition (NO/OPER), and Subgroup.
    Returns (sample_id, condition, subgroup).
    """
    subject_id, condition = parse_filename(filename)

    if subject_id is None:
        subject_id = 'Unknown'

    subgroup = 'Unassigned'
    if condition == 'NO':
        subgroup = 'NON'
    elif condition == 'OPER':
        if subject_id in tfl_ids:
            subgroup = 'TFL'
        elif subject_id in msc_ids:
            subgroup = 'MSC'

    return subject_id, condition, subgroup


# ============================================================================
# DATA VALIDATION & NORMALIZATION (from utils.py)
# ============================================================================

def validate_raw_data(df: pd.DataFrame) -> List[str]:
    """Validate that a raw data DataFrame has required columns and valid values."""
    errors = []
    required_cols = ['Crossheadmm', 'LoadN']
    for col in required_cols:
        if col not in df.columns:
            if col == 'LoadN' and 'LoadkN' in df.columns:
                continue
            errors.append(f'Missing required column: {col}')

    if errors:
        return errors

    if len(df) == 0:
        errors.append('Data file is empty')
        return errors

    if 'Crossheadmm' in df.columns:
        if df['Crossheadmm'].isna().all():
            errors.append('All displacement values are missing')
        elif (df['Crossheadmm'] < 0).any():
            errors.append('Negative displacement values detected')

    load_col = 'LoadN' if 'LoadN' in df.columns else 'LoadkN'
    if load_col in df.columns:
        if df[load_col].isna().all():
            errors.append('All load values are missing')

    if len(df) < 10:
        errors.append(f'Insufficient data points: {len(df)} < 10')

    return errors


def normalize_load_column(df: pd.DataFrame) -> pd.DataFrame:
    """Ensure DataFrame has LoadN column in Newtons."""
    df = df.copy()
    if 'LoadN' not in df.columns and 'LoadkN' in df.columns:
        df['LoadN'] = df['LoadkN'] * 1000
    return df


def safe_trapezoid(y: np.ndarray, x: np.ndarray) -> float:
    """Compute area under curve using trapezoidal rule."""
    try:
        return float(np.trapezoid(y, x))  # NumPy >= 2.0
    except AttributeError:
        return float(np.trapz(y, x))       # NumPy < 2.0


# ============================================================================
# STIFFNESS ANALYSIS (from analysis_pipeline.py)
# ============================================================================

def find_best_stiffness(
    x: np.ndarray,
    y: np.ndarray,
    window_size: int,
    r2_threshold: float
) -> Tuple[float, float, float, int, int]:
    """
    Scan the curve to find the stiffest linear region using a sliding window.
    Returns (slope, intercept, r_squared, start_idx, end_idx).
    """
    best_r2 = -np.inf
    best_slope = 0.0
    best_params = (np.nan, np.nan, np.nan, 0, 0)

    x = np.asarray(x, dtype=np.float64)
    y = np.asarray(y, dtype=np.float64)

    n_points = len(x)
    if n_points < window_size:
        return best_params

    for i in range(n_points - window_size):
        x_win = x[i : i + window_size].reshape(-1, 1)
        y_win = y[i : i + window_size]

        model = LinearRegression().fit(x_win, y_win)
        y_pred = model.predict(x_win)

        r2 = r2_score(y_win, y_pred)
        slope = float(model.coef_[0])
        intercept = float(model.intercept_)

        if r2 >= r2_threshold:
            if slope > best_slope:
                best_slope = slope
                best_params = (slope, intercept, r2, i, i + window_size)

        if r2 > best_r2 and best_slope == 0:
            best_r2 = r2
            best_params = (slope, intercept, r2, i, i + window_size)

    return best_params


# ============================================================================
# BATCH PROCESSING (from analysis_pipeline.py)
# ============================================================================

def process_all_files(
    data_dir: Path,
    tfl_ids: List[str],
    msc_ids: List[str],
    r2_threshold: float,
    window_fraction: float,
    min_window: int,
    excluded_samples: Optional[List[str]] = None
) -> Optional[pd.DataFrame]:
    """Process all CSV files in the data folder and return analysis results."""
    if not data_dir.exists():
        print(f'ERROR: Folder not found: {data_dir}')
        return None

    excluded = set(excluded_samples) if excluded_samples else set()

    files = sorted([f for f in data_dir.iterdir() if f.suffix.lower() == '.csv'])
    data_records: List[Dict] = []

    print(f'Found {len(files)} files. Excluded samples: {sorted(excluded) if excluded else "none"}')

    for file_path in files:
        filename = file_path.name
        try:
            s_id, cond, sub = classify_sample(filename, tfl_ids, msc_ids)

            # Skip excluded samples
            if s_id in excluded:
                print(f'  SKIP {filename}: sample {s_id} is excluded.')
                continue

            if cond == 'Unknown':
                print(f'  SKIP {filename}: Could not determine NO/OPER.')
                continue

            df = pd.read_csv(file_path)

            validation_errors = validate_raw_data(df)
            if validation_errors:
                print(f'  SKIP {filename}: {"; ".join(validation_errors)}')
                continue

            df = normalize_load_column(df)
            y_col = 'LoadN'

            # Truncate at max load (failure point)
            max_idx = df[y_col].idxmax()
            df_trunc = df.iloc[: max_idx + 1].copy()
            x = df_trunc['Crossheadmm'].values
            y = df_trunc[y_col].values

            # Calculate energy (area under curve)
            energy_mJ = safe_trapezoid(y, x)

            # Find stiffness using sliding window
            window_span = max(min_window, int(len(x) * window_fraction))
            slope, intercept, r2, idx_start, idx_end = find_best_stiffness(
                x, y, window_span, r2_threshold
            )

            data_records.append({
                'Filename': filename,
                'SampleID': s_id,
                'Subgroup': sub,
                'MaxLoad_N': float(df[y_col].max()),
                'Stiffness_N_mm': slope,
                'Energy_mJ': energy_mJ,
                'R2_Score': r2,
                'Linear_Start_Idx': idx_start,
                'Linear_End_Idx': idx_end
            })

        except Exception as e:
            print(f'  ERROR {filename}: {e}')

    return pd.DataFrame(data_records)


# ============================================================================
# STATISTICS (from analysis_pipeline.py)
# ============================================================================

def generate_statistics(metadata: pd.DataFrame) -> pd.DataFrame:
    """Generate group statistics from the analysis results."""
    def list_ids(series: pd.Series) -> str:
        return ', '.join(sorted(series.unique()))

    stats = metadata.groupby('Subgroup').agg(
        MaxLoad_Mean=('MaxLoad_N', 'mean'),
        MaxLoad_Std=('MaxLoad_N', 'std'),
        Stiffness_Mean=('Stiffness_N_mm', 'mean'),
        Stiffness_Std=('Stiffness_N_mm', 'std'),
        Energy_Mean=('Energy_mJ', 'mean'),
        Energy_Std=('Energy_mJ', 'std'),
        Count=('SampleID', 'count'),
        Sample_List=('SampleID', list_ids)
    ).round(2)

    return stats


print('\nAll functions defined. Ready to run the pipeline.')

## Cell 2: Understanding the Classification

The `classify_sample()` function determines which group a sample belongs to based on its filename.

**Logic:**
- Filenames contain: `SSP_YYYY-MM-DD_SubjectID_Condition.csv`
- Condition is either `NO` (non-operated control) or `OPER` (operated)
- Non-operated samples are always in the `NON` (control) group
- Operated samples are assigned to `TFL` or `MSC` based on the hardcoded group IDs

In [ ]:
# Demonstrate the classification function
test_files = [
    'SSP_2025-03-17_D1_NO.csv',
    'SSP_2025-03-17_D1_OPER.csv',
    'SSP_2025-03-20_D9_OPER.csv'
]

print('Classification examples:')
print('-' * 60)
for f in test_files:
    sample_id, condition, subgroup = classify_sample(f, TFL_IDS, MSC_IDS)
    print(f'{f}')
    print(f'  -> Sample: {sample_id}, Condition: {condition}, Group: {subgroup}')
    print()

## Cell 3: Run the Analysis Pipeline

The analysis pipeline processes all CSV files and calculates:

1. **Max Load (N)** - Ultimate failure load
2. **Stiffness (N/mm)** - Slope of the stiffest linear region (using sliding window)
3. **Energy (mJ)** - Area under the load-displacement curve (work to failure)

**Stiffness Calculation:**
We use a sliding window approach to find the region with:
- Highest slope (stiffness)
- Good linearity (R² > 0.99)

In [ ]:
# Run the analysis pipeline (uses ACTIVE lists, which respect EXCLUDED_SAMPLES)
metadata = process_all_files(
    data_dir=DATA_FOLDER,
    tfl_ids=ACTIVE_TFL,
    msc_ids=ACTIVE_MSC,
    r2_threshold=STIFFNESS_R2_THRESHOLD,
    window_fraction=STIFFNESS_WINDOW_FRACTION,
    min_window=STIFFNESS_MIN_WINDOW,
    excluded_samples=EXCLUDED_SAMPLES
)

if metadata is not None:
    print(f'\nProcessed {len(metadata)} samples successfully.')
    display(metadata.head(10))

## Cell 4: Group Statistics

Generate summary statistics for each treatment group:
- **NON** - Non-operated control (intact rotator cuff)
- **TFL** - Tensor fascia lata allograft
- **MSC** - TFL + Mesenchymal stem cells

In [ ]:
# Generate statistics
stats = generate_statistics(metadata)
display(stats)

# Save results to CSV
detail_path = RESULTS_FOLDER / 'Experiment_Master_Log_Detailed.csv'
metadata.to_csv(detail_path, index=False)
print(f'\nSaved detailed results to: {detail_path}')

stats_path = RESULTS_FOLDER / 'Group_Statistics_Detailed.csv'
stats.to_csv(stats_path)
print(f'Saved group statistics to: {stats_path}')

## Cell 5: Visualization - Group Comparison Bar Charts

Compare the three groups across all metrics with error bars showing standard deviation.

In [ ]:
# Create comparison bar charts
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

groups = ['NON', 'TFL', 'MSC']
colors = ['#2ecc71', '#3498db', '#e74c3c']
metrics = ['MaxLoad_N', 'Stiffness_N_mm', 'Energy_mJ']
titles = ['Max Load (N)', 'Stiffness (N/mm)', 'Energy (mJ)']

for i, (metric, title) in enumerate(zip(metrics, titles)):
    means = [metadata[metadata['Subgroup'] == g][metric].mean() for g in groups]
    stds = [metadata[metadata['Subgroup'] == g][metric].std() for g in groups]
    counts = [len(metadata[metadata['Subgroup'] == g]) for g in groups]
    
    bars = axes[i].bar(groups, means, color=colors, alpha=0.8, edgecolor='black')
    axes[i].errorbar(groups, means, yerr=stds, fmt='none', color='black', capsize=5)
    
    # Add sample counts
    for j, bar in enumerate(bars):
        std_val = stds[j] if not np.isnan(stds[j]) else 0
        axes[i].text(
            bar.get_x() + bar.get_width() / 2,
            bar.get_height() + std_val + 5,
            f'n={counts[j]}',
            ha='center', va='bottom', fontsize=10
        )
    
    axes[i].set_title(title, fontsize=14, fontweight='bold')
    axes[i].set_ylabel(title)
    axes[i].set_xlabel('Treatment Group')

plt.tight_layout()

# Save plot
plot_path = RESULTS_FOLDER / 'Combined_Group_Plots.png'
plt.savefig(plot_path, dpi=150, bbox_inches='tight')
print(f'Saved plot to: {plot_path}')
plt.show()

## Cell 6: Individual Load-Displacement Curves

Plot the raw load-displacement curves for each group to visualize the mechanical behavior.

In [ ]:
def plot_group_curves(group_name, group_df, ax, colormap_name='viridis'):
    """Plot load-displacement curves for a group."""
    cmap = plt.get_cmap(colormap_name)
    n_samples = len(group_df)
    
    for idx, (_, row) in enumerate(group_df.iterrows()):
        filename = row['Filename']
        filepath = DATA_FOLDER / filename
        
        if not filepath.exists():
            continue
            
        df = pd.read_csv(filepath)
        if 'LoadN' not in df.columns and 'LoadkN' in df.columns:
            df['LoadN'] = df['LoadkN'] * 1000
        
        # Truncate at max load
        max_idx = df['LoadN'].idxmax()
        df_trunc = df.iloc[:max_idx + 1]
        
        color = cmap(idx / max(n_samples - 1, 1))
        ax.plot(
            df_trunc['Crossheadmm'], 
            df_trunc['LoadN'],
            color=color,
            alpha=0.7,
            linewidth=1.5,
            label=row['SampleID']
        )
    
    ax.set_title(f'{group_name} Group (n={n_samples})', fontsize=14, fontweight='bold')
    ax.set_xlabel('Displacement (mm)')
    ax.set_ylabel('Load (N)')
    ax.legend(loc='upper left', fontsize=8, ncol=2)
    ax.grid(True, alpha=0.3)


# Create subplots for each group
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

group_configs = [
    ('NON', 'Greens'),
    ('TFL', 'Blues'),
    ('MSC', 'Reds')
]

for ax, (group_name, colormap) in zip(axes, group_configs):
    group_df = metadata[metadata['Subgroup'] == group_name]
    plot_group_curves(group_name, group_df, ax, colormap)

plt.tight_layout()

# Save individual group plots
for group_name, colormap in group_configs:
    fig_single, ax_single = plt.subplots(figsize=(10, 6))
    group_df = metadata[metadata['Subgroup'] == group_name]
    plot_group_curves(group_name, group_df, ax_single, colormap)
    plot_path = RESULTS_FOLDER / f'Plot_{group_name}.png'
    fig_single.savefig(plot_path, dpi=150, bbox_inches='tight')
    plt.close(fig_single)
    print(f'Saved: {plot_path}')

plt.show()

## Cell 7: Interactive Plots (Plotly)

If Plotly is available, create interactive plots for detailed exploration.

In [ ]:
if PLOTLY_AVAILABLE:
    # Create interactive comparison chart
    fig = make_subplots(
        rows=1, cols=3,
        subplot_titles=['Max Load (N)', 'Stiffness (N/mm)', 'Energy (mJ)']
    )
    
    colors_plotly = {'NON': '#2ecc71', 'TFL': '#3498db', 'MSC': '#e74c3c'}
    
    for i, metric in enumerate(['MaxLoad_N', 'Stiffness_N_mm', 'Energy_mJ']):
        for group in ['NON', 'TFL', 'MSC']:
            group_data = metadata[metadata['Subgroup'] == group][metric]
            fig.add_trace(
                go.Box(
                    y=group_data,
                    name=group,
                    marker_color=colors_plotly[group],
                    showlegend=(i == 0)
                ),
                row=1, col=i+1
            )
    
    fig.update_layout(
        title='Biomechanical Properties by Treatment Group',
        height=500,
        showlegend=True
    )
    fig.show()
else:
    print('Plotly not available. Skipping interactive plots.')

## Cell 8: Stiffness Visualization (All Subjects)

Show the linear region used for stiffness calculation on **every sample**, organized by group.

In [ ]:
def plot_stiffness_region(sample_row):
    """Plot a sample's curve with the linear region highlighted."""
    filename = sample_row['Filename']
    filepath = DATA_FOLDER / filename
    
    df = pd.read_csv(filepath)
    if 'LoadN' not in df.columns and 'LoadkN' in df.columns:
        df['LoadN'] = df['LoadkN'] * 1000
    
    # Truncate at max load
    max_idx = df['LoadN'].idxmax()
    df_trunc = df.iloc[:max_idx + 1]
    
    x = df_trunc['Crossheadmm'].values
    y = df_trunc['LoadN'].values
    
    # Get linear region indices
    start_idx = int(sample_row['Linear_Start_Idx'])
    end_idx = int(sample_row['Linear_End_Idx'])
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Plot full curve
    ax.plot(x, y, 'b-', linewidth=2, label='Load-Displacement Curve')
    
    # Highlight linear region
    ax.plot(x[start_idx:end_idx], y[start_idx:end_idx], 'r-', 
            linewidth=3, label=f'Linear Region (R\u00b2={sample_row["R2_Score"]:.4f})')
    
    # Add linear fit line
    slope = sample_row['Stiffness_N_mm']
    x_fit = x[start_idx:end_idx]
    y_fit_start = y[start_idx]
    y_fit = y_fit_start + slope * (x_fit - x_fit[0])
    ax.plot(x_fit, y_fit, 'g--', linewidth=2, 
            label=f'Stiffness = {slope:.1f} N/mm')
    
    ax.set_xlabel('Displacement (mm)', fontsize=12)
    ax.set_ylabel('Load (N)', fontsize=12)
    ax.set_title(f'{filename}\nMax Load: {sample_row["MaxLoad_N"]:.1f} N, '
                 f'Energy: {sample_row["Energy_mJ"]:.1f} mJ', fontsize=12)
    ax.legend(loc='upper left')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    return fig


# Plot ALL subjects organized by group
for group in ['NON', 'TFL', 'MSC']:
    group_df = metadata[metadata['Subgroup'] == group]
    n_samples = len(group_df)
    
    if n_samples == 0:
        continue
    
    print(f"\n{'='*60}")
    print(f'{group} GROUP ({n_samples} samples)')
    print(f"{'='*60}")
    
    for idx, (_, sample) in enumerate(group_df.iterrows()):
        print(f"\n[{idx+1}/{n_samples}] {sample['SampleID']}:")
        fig = plot_stiffness_region(sample)
        plt.show()
        plt.close(fig)  # Close to free memory

## Summary

This notebook demonstrates the biomechanical analysis pipeline for the SSP study.

**Key Findings:**
- Non-operated (NON) samples show the highest mechanical properties (intact tissue)
- MSC group shows improved properties compared to TFL-only group
- Both operated groups show significantly reduced properties vs. intact controls

**Files Generated:**
- `Results/Experiment_Master_Log_Detailed.csv` - All sample data
- `Results/Group_Statistics_Detailed.csv` - Summary statistics
- `Results/Combined_Group_Plots.png` - Comparison bar charts
- `Results/Plot_NON.png`, `Plot_TFL.png`, `Plot_MSC.png` - Individual group curves